In [1]:
# 패키지 로드
import cv2
import math as m
import mediapipe as mp
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
font = cv2.FONT_HERSHEY_SIMPLEX

#Dataset 경로 지정
root_dir = 'C:/Users/SJ/AI1/Final Project/train/squat_down/'

#불러올 파일의 확장자 지정(여기서는 이미지 파일만 로드)
img_path_list = []
possible_img_extension = ['.jpg', '.jpeg', '.JPG', '.bmp', '.png']

#Dataset의 경로를 리스트 형태로 가져옴
for (root, dirs, files) in os.walk(root_dir):
    if len(files) > 0:
        for file_name in files:
            if os.path.splitext(file_name)[1] in possible_img_extension:
                img_path = root + '/' + file_name
                
                # 경로에서 \를 모두 /로 바꿔줘야 오류가 생기지 않음
                img_path = img_path.replace('\\', '/') # \는 \\로 나타내야함         
                img_path_list.append(img_path)

# 관절 각도를 저장할 리스트 선언
upper_body_angle_list=[]
knee_angle_list=[]
knee_tiptoe_diff_list=[]

# 각도 계산 함수
def calculate_angle(a,b,c):
    a = np.array(a) # 첫 번째 점의 좌표
    b = np.array(b) # 두 번째 점의 좌표
    c = np.array(c) # 세 번째 점의 좌표
    
    # 역탄젠트를 이용해 라디언값을 구해줌
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    # 라디언값을 도단위로 변환
    angle = np.abs(radians*180.0/np.pi)
    
    # 우리는 스쿼트를 하는 신체 각도를 구할 것이므로 180도 이하의 값을 보이도록 보정
    if angle >180.0:
        angle = 360-angle
        
    return angle 


#mediapipe 및 cv를 사용하여 이미지에서 관절 좌표 추출

with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        min_detection_confidence=0.5) as pose:
    
# 이미지 파일 경로를 타고 이미지 가져오기
    for idx, file in enumerate(img_path_list):
        image = cv2.imread(file)
        image_height, image_width, _ = image.shape
        h, w = image.shape[:2] 
        
        # 처리 전 opencv를 통해 불러온 이미지를 BGR 순서에서 RGB 순서로 변환
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if not results.pose_landmarks:
            continue

        landmarks = results.pose_landmarks.landmark
            
        # mediapipe를 통해 관절 좌표 로드
        left_shoulder = [int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x*w),
                         int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y*h)]
        left_hip = [int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x*w),
                    int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y*h)]
        left_knee = [int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x*w),
                     int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y*h)]
        left_ankle = [int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x*w),
                      int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y*h)]
        left_heel = [int(landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x*w),
                     int(landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y*h)]
        left_foot_index = [int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x*w),
                           int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y*h)]
        right_shoulder = [int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x*w),
                          int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y*h)]
        
        # 정확한 자세인지 확인하기 위해 허리, 무릎의 각도 및 발끝과 무릎 사이의 거리 계산
        upper_body_angle = int((calculate_angle(left_shoulder, left_hip, left_knee)))
        knee_angle = int((calculate_angle(left_hip, left_knee,left_heel)))
        knee_tiptoe_diff = int(left_foot_index[0]-left_knee[0])
        
        # 데이터별 각도를 각 리스트에 append
        upper_body_angle_list.append(upper_body_angle)
        knee_angle_list.append(knee_angle)
        knee_tiptoe_diff_list.append(knee_tiptoe_diff) 

#스쿼트 각도 계산에 필요한 허리 각도, 무릎 각도, 무릎과 발끝의 좌표 차이 변수 선언
upper_body_angle_avg=0
knee_angle_avg=0
knee_tiptoe_diff_avg=0


In [2]:
#리스트의 평균을 구하는 함수 생성
def list_average(list):
    avg=0
    sum=0
    for i in list:
        sum+=i
        avg=int(sum/len(list))
    return avg

In [3]:
#허리 각도, 무릎 각도, 무릎과 발끝의 좌표 차이 평균 계산
upper_body_angle_avg=list_average(upper_body_angle_list)
knee_angle_avg=list_average(knee_angle_list)
knee_tiptoe_diff_avg=list_average(knee_tiptoe_diff_list)

In [4]:
upper_body_angle_avg

83

In [5]:
knee_angle_avg

89

In [6]:
knee_tiptoe_diff_avg

-7

In [7]:
# 우리는 웹캠을 활용하므로, 카메라가 신체 측면을 찍고 있는지 확인하기 위해 거리 계산 함수 선언
def findDistance(x1, y1, x2, y2):
    dist = m.sqrt((x2-x1)**2+(y2-y1)**2)
    return dist

In [8]:
# 시각화를 위해 색상 변수 선언
white = (255, 255, 255)
red = (50, 50, 255)
green = (127, 255, 0)
light_green = (127, 233, 100)


In [36]:
#cv를 통해 동영상 불러오기. (0)은 카메라를 통해 정보를 가져옴
cap = cv2.VideoCapture(0) 

# 초기 변수 설정. 카운터는 0번, 스쿼트 포지션은 up, 상태는 not error로 지정
counter = 0
position = "up"
state = "not error"
aligned_state = 'not aligned'

# mediapipe 및 cv를 사용하여 웹캠을 통해 불러온 영상의 관절 좌표 추출 및 분석
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # 처리 전 opencv를 통해 불러온 이미지를 BGR 순서에서 RGB 순서로 변환
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False #이미지 다시쓰기
        h, w = image.shape[:2]      
        # 탐지하기
        results = pose.process(image)
    
        # cv2를 통해 출력하기 위해 RGB 순서의 이미지를 다시 BGR로 변환
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Body Posture Analysis 시작!
        try:
            landmarks = results.pose_landmarks.landmark
            
            # mediapipe를 통해 관절 좌표 로드
            left_shoulder = [int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x*w),
                             int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y*h)]
            left_hip = [int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x*w),
                        int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y*h)]
            left_knee = [int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x*w),
                         int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y*h)]
            left_ankle = [int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x*w),
                          int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y*h)]
            left_heel = [int(landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x*w),
                         int(landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y*h)]
            left_foot_index = [int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x*w),
                               int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y*h)]
            right_shoulder = [int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x*w),
                              int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y*h)]       
            # 정확한 자세인지 확인하기 위해 허리, 무릎의 각도 및 발끝과 무릎 사이의 거리 계산
            upper_body_angle = int((calculate_angle(left_shoulder, left_hip, left_knee)))
            knee_angle = int((calculate_angle(left_hip, left_knee,left_heel)))
            knee_tiptoe_diff = int(left_foot_index[0]-left_knee[0])
            
            #신체 측면이 촬영되고 있는지 확인하기 위해 양쪽 어깨의 거리를 계산
            offset = findDistance(left_shoulder[0], left_shoulder[1], right_shoulder[0], right_shoulder[1])

            # 양쪽 어깨의 거리가 일정 간격 이하면 Body Posture Analysis 시작
            if offset < 30:
                aligned_state = 'aligned'

                # 상태 지정
                # 'up' 상태에서 허리 및 무릎 각도가 Dataset의 평균값 이하로 내려갈 경우 'down'으로 상태 전환
                if (upper_body_angle<= upper_body_angle_avg or knee_angle <= knee_angle_avg) and position=='up':
                    position = "down"
            
                # 'down' 상태에서 허리 및 무릎 각도가 일어선 수준으로 올라갈 경우 'up'으로 상태 전환 후 스쿼트 카운트
                elif upper_body_angle >= 170 and knee_angle >= 170 and position=='down':
                    position="up"
                    counter +=1

                #자세 정확도 판별
                # 허리 및 무릎의 각도, 발의 위치가 정상 자세와 일정 이상의 차이가 생길 경우 'error' 상태로 저장
                if ((upper_body_angle<= upper_body_angle_avg+10 or knee_angle <= knee_angle_avg+10)
                    and (upper_body_angle <= upper_body_angle_avg-20 or knee_angle <= knee_angle_avg-20
                    or knee_tiptoe_diff < knee_tiptoe_diff_avg-5)) :
                    state = "error"
                
                # 정확한 자세일 때는 'correct' 상태로 저장
                elif (upper_body_angle <= upper_body_angle_avg+10 and upper_body_angle >= upper_body_angle_avg-10
                      and knee_angle <= knee_angle_avg+10 and knee_angle <= knee_angle_avg-10
                     ) :
                    state = "correct"
                
                else : 
                    state = "not error"

            #양쪽 어깨가 정면을 보고 있을 경우 Not Aligned 문구 표시
            else:
                aligned_state = 'not aligned'

        except:
            pass

        
        #상태 표시창 배경 그리기
        cv2.rectangle(image, (0,0), (800,85), (227,235,255), -1)
        
        if aligned_state == 'aligned':
            cv2.putText(image, str(int(offset)) + ' Aligned', (470, 50), font, 0.6, green, 2)
        elif aligned_state == 'not aliged':
            cv2.putText(image, str(int(offset)) + ' Not Aligned', (470, 50), font, 0.6, red, 2, cv2.LINE_AA)
        
        # 스쿼트 카운트 표시
        cv2.putText(image, 'Count', (15,20), font, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10,60), font, 1.5, (0,0,0), 2, cv2.LINE_AA)
        
        # 스쿼트 스테이지와 몸의 각도 표시
        cv2.putText(image, 'position', (120,20), font, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, position, (115,60), font, 1.5, (0,0,0), 2, cv2.LINE_AA)

        cv2.putText(image, 'upper_body_angle : '+ str(upper_body_angle), (280,20), font, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, 'knee_angle : '+ str(knee_angle), (280,40), font, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, 'knee : '+ str(knee_tiptoe_diff), (280,60), font, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, 'state : '+ str(state),  (280,80), font, 0.5, (0,0,0), 1, cv2.LINE_AA)
        
        # 스쿼트 자세가 정확할 경우 초록색으로 자세 표시
        if state == "correct":
            cv2.putText(image, str(int(upper_body_angle)), (left_hip[0] + 6, left_hip[1]), font, 0.9, light_green, 2)
            cv2.putText(image, str(int(knee_angle)), (left_knee[0] + 6, left_knee[1]), font, 0.9, light_green, 2)
        
            cv2.circle(image, (left_shoulder[0], left_shoulder[1]), 7, green, -1)
            cv2.circle(image, (left_hip[0], left_hip[1]), 7, green, -1)
            cv2.circle(image, (left_knee[0], left_knee[1]), 7, green, -1)
            cv2.circle(image, (left_ankle[0], left_ankle[1]), 7, green, -1)
            cv2.circle(image, (left_foot_index[0], left_foot_index[1]), 7, green, -1)

            cv2.line(image, (left_shoulder[0], left_shoulder[1]), (left_hip[0], left_hip[1]), green, 4)
            cv2.line(image, (left_hip[0], left_hip[1]), (left_knee[0], left_knee[1]), green, 4)
            cv2.line(image, (left_knee[0], left_knee[1]), (left_ankle[0], left_ankle[1]), green, 4)
            cv2.line(image, (left_ankle[0], left_ankle[1]), (left_foot_index[0], left_foot_index[1]), green, 4)
        
        #스쿼트 자세가 부정확할 경우 빨간색으로 자세 표시
        elif state == 'error':
            cv2.putText(image, str(int(upper_body_angle)), (left_hip[0] + 6, left_hip[1]), font, 0.9, red, 2)
            cv2.putText(image, str(int(knee_angle)), (left_knee[0] + 6, left_knee[1]), font, 0.9, red, 2)

            cv2.circle(image, (left_shoulder[0], left_shoulder[1]), 7, red, -1)
            cv2.circle(image, (left_hip[0], left_hip[1]), 7, red, -1)
            cv2.circle(image, (left_knee[0], left_knee[1]), 7, red, -1)
            cv2.circle(image, (left_ankle[0], left_ankle[1]), 7, red, -1)
            cv2.circle(image, (left_foot_index[0], left_foot_index[1]), 7, red, -1)
            
            cv2.line(image, (left_shoulder[0], left_shoulder[1]), (left_hip[0], left_hip[1]), red, 4)
            cv2.line(image, (left_hip[0], left_hip[1]), (left_knee[0], left_knee[1]), red, 4)
            cv2.line(image, (left_knee[0], left_knee[1]), (left_ankle[0], left_ankle[1]), red, 4)
            cv2.line(image, (left_ankle[0], left_ankle[1]), (left_foot_index[0], left_foot_index[1]), red, 4)
            
        #스쿼트 과정의 자세가 정상 범위일 경우 흰색으로 자세 표시
        elif state == 'not error':
            cv2.putText(image, str(int(upper_body_angle)), (left_hip[0] + 6, left_hip[1]), font, 0.9, white, 2)
            cv2.putText(image, str(int(knee_angle)), (left_knee[0] + 6, left_knee[1]), font, 0.9, white, 2)

            cv2.circle(image, (left_shoulder[0], left_shoulder[1]), 7, white, -1)
            cv2.circle(image, (left_hip[0], left_hip[1]), 7, white, -1)
            cv2.circle(image, (left_knee[0], left_knee[1]), 7, white, -1)
            cv2.circle(image, (left_ankle[0], left_ankle[1]), 7, white, -1)
            cv2.circle(image, (left_foot_index[0], left_foot_index[1]), 7, white, -1)
            
            cv2.line(image, (left_shoulder[0], left_shoulder[1]), (left_hip[0], left_hip[1]), white, 4)
            cv2.line(image, (left_hip[0], left_hip[1]), (left_knee[0], left_knee[1]), white, 4)
            cv2.line(image, (left_knee[0], left_knee[1]), (left_ankle[0], left_ankle[1]), white, 4)
            cv2.line(image, (left_ankle[0], left_ankle[1]), (left_foot_index[0], left_foot_index[1]), white, 4)
             
        # pose detection이 적용된 웹캠 화면을 출력
        cv2.imshow('Real-Time Squat Counter', image)
        if cv2.waitKey(10) & 0xFF == 27: #esc 키 누르면 닫음
            break

    cap.release()
    cv2.destroyAllWindows()